# KRAS: Effects of Mutation

<b> Standard imports for playing with and plotting data frames. </b>

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

<b> Import CPTAC </b>

In [2]:
import CPTAC

Loading CPTAC data:
Loading Dictionary...
Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter CPTAC.embargo() to open the webpage for more details.


In [3]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

<b> List of proteins that interact with KRAS (according to STRING) </b>

In [4]:
protList = ['EGRF', 'PIK3CA', 'ERBB2', 'ARAF', 'BRAF', 'MAPK1', 'MAPK3', 'MAP2K1', 'SOS1', 'RAF1']

### Proteome abundance of proteins that interact with KRAS

In [5]:
gene = 'KRAS'
sigList = [];

pcutoff = 0.05/len(protList)
print(pcutoff)

genedf = somatic_mutations[gene].to_frame()
for protein in protList:
    if protein in proteomics.columns:
        proteindf = proteomics[protein].to_frame()
        proteindfName = protein + " protein"
        proteindf.columns = [proteindfName]
        cross = genedf.add(proteindf, fill_value=0).dropna(axis=0)
        mutated = cross.loc[cross[gene] == 1.0]
        wt = cross.loc[cross[gene] == 0.0]
        ttest = scipy.stats.ttest_ind(mutated[proteindfName], wt[proteindfName])
        if ttest[1] <= pcutoff:
            sigList.append(protein)
            print("Test for " + protein + ": ")
            print(ttest)

0.005


KeyError: 'KRAS'

<b> Print out the list of proteins  </b>

In [ ]:
print(sigList)

### Phosphoproteome abundance of interacting proteins

In [ ]:
phosProtList = ['EGRF', 'PIK3CA', 'ERBB', 'ARAF', 'BRAF', 'MAPK', 'MAPK', 'MAP2K', 'SOS', 'RAF']

In [ ]:
gene = 'KRAS'
genedf = somatic_mutations[gene].to_frame()
sigPhosResults = [];

for protein in phosProtList:
    sites = phos.filter(regex=protein)
    genedf = genedf.add(sites, fill_value=0)

mutated = genedf.loc[genedf[gene] == 1.0]
wt = genedf.loc[genedf[gene] == 0.0]

pcutoff = 0.05 / len(genedf.columns)
print(pcutoff)

for loc in genedf.columns:
    if not loc == gene:
        mutsitedf = mutated[[gene, loc]].dropna()
        wtsitedf = wt[[gene, loc]].dropna()
        ttest = scipy.stats.ttest_ind(mutsitedf[loc], wtsitedf[loc])
        if(ttest[1] <= pcutoff):
            sigPhosResults.append(loc)
            print('Results for ' + loc + ': ')
            print(ttest)
 